# 共通処理

In [ ]:
import os
import sys

import h5py

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


In [ ]:
import json

try:
    with open('setting.json', 'r') as file:
        setting = json.load(file)
except FileNotFoundError:
    print("先にset_run.ipynbで解析runを設定してください")

hdf5_path = setting['hdf5_path']
hdf5_path

run_name = hdf5_path.split('/')[-1].split('.')[0]

hdf5_path, run_name

In [ ]:
from model.File.HDF5 import HDF5

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [ ]:
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['mathtext.fontset'] = 'cm'     #数式用のフォントejavuserif" or "cm"
plt.rcParams['xtick.direction'] = 'in'      #x軸の目盛線 #内in')か外向き('out')か双方向か('inout')
plt.rcParams['ytick.direction'] = 'in'      #y軸の目盛線 #内in')か外向き('out')か双方向か('inout')
plt.rcParams['xtick.major.width'] = 1.0     #x軸主目盛り線の線幅
plt.rcParams['ytick.major.width'] = 1.0     #y軸主目盛り線の線幅
plt.rcParams['font.size'] = 18               #フォントの大きさ
plt.rcParams['axes.linewidth'] = 0.8        #軸の線幅edge linewidth。囲みの太さ
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = (8, 6)


In [ ]:
# hdf5内のlayer pathを取得する
with h5py.File(hdf5_path, 'r') as f_read:
    hdf = HDF5(f=f_read)

In [ ]:
def return_hdf5_data(hdf5_path: str, *, query: str, shape=None):
    with h5py.File(hdf5_path, 'r') as f_read:
        hdf = HDF5(f=f_read)
        
        to_data = hdf.search_data_path(query=query)
        
        if type(to_data) is str:
            return hdf.return_data(data_path=to_data, shape=shape)
        elif type(to_data) is list:
            raise Exception("複数のlayer pathが見つかりました。一括で返して欲しい場合は実装してください。")
        else:
            raise Exception("layer pathが見つかりませんでした。")

# データの取得

In [ ]:
# 温度データ
T_arr = return_hdf5_data(hdf5_path, query='T_arr')
frame_num = return_hdf5_data(hdf5_path, query='spe/frame_num')
time_arr = return_hdf5_data(hdf5_path, query='spe/time_arr')
fps = return_hdf5_data(hdf5_path, query='spe/fps')
exposure_ms = return_hdf5_data(hdf5_path, query='spe/exposure_ms')

In [ ]:
# レーザー
laser_time_arr = return_hdf5_data(hdf5_path, query='laser/time_arr')
laser_power_arr = return_hdf5_data(hdf5_path, query='power')

In [ ]:
# スペクトル
spectra = return_hdf5_data(hdf5_path, query='spectra')
wl_arr = return_hdf5_data(hdf5_path, query='spe/wl_arr')
pixel_arr = return_hdf5_data(hdf5_path, query='spe/pixel_arr')

# 波長の端の方は邪魔なので切り取っておく
spectra = spectra[:,:,50:450] # [frame, postion_pixel, wavelength_pixel]
wl_arr = wl_arr[50:450]

In [ ]:
# 生の輻射データ
radiation = return_hdf5_data(hdf5_path, query='spe/img')

In [ ]:
# XRDに合わせるために、XRDのfpsなどを取得
xrd_fps = return_hdf5_data(hdf5_path, query='nxs/fps')
xrd_exposure_ms = return_hdf5_data(hdf5_path, query='nxs/exposure_ms')
xrd_frame_num = return_hdf5_data(hdf5_path, query='nxs/frame_num')

In [ ]:
# pixelを長さに変換するスケールを取得

with open('setting.json', 'r') as file:
    setting = json.load(file)

pixel_to_um = setting['pixel_to_um']
pixel_to_um

In [ ]:
# laser径を取得
laser_diameter_um = setting['laser_params']['laser_diameter_um']

laser_diameter_pixel = round(laser_diameter_um / pixel_to_um)

print(f"{laser_diameter_um = } um, {laser_diameter_pixel = } pixel")

In [ ]:
# pixel数入れるの忘れたので設定する
pixel_num = pixel_arr[-1] + 1
pixel_num = 512 # あとでコメントアウト
pixel_num

# データ加工

In [ ]:
# 生の輻射の各frameの最大強度
max_radi_frame_list = []
for frame in tqdm(range(frame_num)):
    max_radi_frame_list.append(np.max(radiation[frame,:,:]))
max_radi_frame_arr = np.array(max_radi_frame_list)

# 校正スペクトルの各frameの最大強度
max_spectra_frame_list = []
for frame in tqdm(range(frame_num)):
    max_spectra_frame_list.append(np.max(spectra[frame,:,:]))
max_spectra_frame_arr = np.array(max_spectra_frame_list)

# 各pos_pixelの最大強度
max_radi_pixel_list = []
for pixel in tqdm(range(pixel_num)):
    max_radi_pixel_list.append(np.max(radiation[:,pixel,:]))
max_radi_pixel_arr = np.array(max_radi_pixel_list)

# 各pos_pixelの最大強度
max_spectra_pixel_list = []
for pixel in tqdm(range(pixel_num)):
    max_spectra_pixel_list.append(np.max(spectra[:,pixel,:]))
max_spectra_pixel_arr = np.array(max_spectra_pixel_list)


# データを眺める

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

ax = axes[0]
ax.plot(max_radi_frame_arr)
ax.set_title('time series')

ax = axes[1]
ax.plot(max_radi_pixel_arr)
ax.set_title('pixel series')

plt.suptitle("spe")
plt.show()
plt.close()

In [ ]:
# calib
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

ax = axes[0]
ax.plot(max_spectra_frame_arr)
ax.set_title('time series')

ax = axes[1]
ax.plot(max_spectra_pixel_arr)
ax.set_title('pixel series')

plt.suptitle("calib")
plt.show()
plt.close()


In [ ]:
# dist
plt.imshow(T_arr, aspect='auto', cmap='jet')
plt.colorbar()
plt.suptitle("T dist")
plt.xlabel("Position pixel")
plt.ylabel("Frame")
plt.show()
plt.close()

# 採用する温度領域(位置、時間)を決める

In [ ]:
# speから決めてる

# 時間
# 最初の1秒で加熱してないことを仮定
# 10 ms露光の場合は 40 frameくらいまで

window = [0, frame_num] # 確認する範囲

max = max_radi_frame_arr[0:40].max()
std = np.std(max_radi_frame_arr[0:40])

threshold = max + std * 5

boundary_frames = []
adopt_flag = False
I_saturation = 65_000
for frame in range(frame_num):
    if adopt_flag is False: # 不採用 -> 採用
        if (max_radi_frame_arr[frame] > threshold) and (max_radi_frame_arr[frame] < I_saturation):
            print(frame, boundary_frames)
            boundary_frames.append(frame)
            adopt_flag = True
    else: # 採用 -> 不採用
        if (max_radi_frame_arr[frame] < threshold) or (max_radi_frame_arr[frame] > I_saturation):
            print(frame, boundary_frames)
            boundary_frames.append(frame)
            adopt_flag = False
print(f"{boundary_frames}")

plt.plot(max_radi_frame_arr)
for i in range(len(boundary_frames)):
    if i % 2 == 0:
        plt.scatter(boundary_frames[i], max_radi_frame_arr[boundary_frames[i]], color='red', s=30, zorder=10)
    else:
        plt.scatter(boundary_frames[i], max_radi_frame_arr[boundary_frames[i]], color='green', s=30, zorder=10)
        
plt.axhline(y=max, color='black', linestyle='--')
plt.axhline(y=threshold, color='r', linestyle='--')
plt.xlim(window[0], window[1])
plt.show()
plt.close()

In [ ]:
# calibから決めてる

# 位置
# 最大強度を取得して、レーザー径の範囲を染める

window = [100, 400] # 確認する範囲
upper_to_right = -1 # pixel: 手で調整。減らす場合はマイナス
lower_to_right = 0 # pixel

upper_center = max_spectra_pixel_arr[0:255].argmax()
lower_center = max_spectra_pixel_arr[256:511].argmax() + 256

upper_left = upper_center - laser_diameter_pixel / 2 + upper_to_right
upper_right = upper_center + laser_diameter_pixel / 2 + upper_to_right
lower_left = lower_center - laser_diameter_pixel / 2 + lower_to_right
lower_right = lower_center + laser_diameter_pixel / 2 + lower_to_right
if laser_diameter_pixel % 2 != 0: # レーザー径pixelが整数にならない場合、適当にずらす
    upper_left = round(upper_left + 0.5)
    upper_right = round(upper_right + 0.5)
    lower_left = round(lower_left + 0.5)
    lower_right = round(lower_right + 0.5)
upper_left = int(upper_left)
upper_right = int(upper_right)
lower_left = int(lower_left)
lower_right = int(lower_right)

print(upper_center, lower_center)

plt.figure(figsize=(10, 5))
plt.plot(max_spectra_pixel_arr)
plt.scatter(upper_center, max_spectra_pixel_arr[upper_center], color='red', s=30, zorder=10)
plt.scatter(lower_center, max_spectra_pixel_arr[lower_center], color='red', s=30, zorder=10)
plt.axvspan(upper_left, upper_right, facecolor='black', alpha=0.3)
plt.axvspan(lower_left, lower_right, facecolor='black', alpha=0.3)
plt.xlim(window[0], window[1])
plt.show()
plt.close()

In [ ]:
# 温度分布で確認
plt.figure(figsize=(10, 5))
plt.imshow(T_arr, aspect='auto', cmap='Grays')
plt.axvspan(xmin=upper_left, xmax=upper_right,
            facecolor='red', alpha=0.1)
plt.axvspan(xmin=lower_left, xmax=lower_right,
            facecolor='red', alpha=0.1)
region_num = int(len(boundary_frames) / 2)
for i in range(region_num):
    i *= 2
    plt.axhspan(ymin=boundary_frames[i], ymax=boundary_frames[i+1],
                facecolor='red', alpha=0.1)
plt.xlim(upper_left - 10, lower_right + 10)
plt.ylim(boundary_frames[0] - 20, boundary_frames[-1] + 20)
plt.show()
plt.close()

# P-T plotで使用する温度配列を作成

In [ ]:
# 採用領域を切り出す
adopt_arr = np.zeros(T_arr.shape)

for i in range(region_num):
    idx = i * 2
    print(f"i: {i}, idx: {idx}")
    start_frame = int(boundary_frames[idx])
    end_frame = int(boundary_frames[idx + 1])
    print(f"start_frame: {start_frame}, end_frame: {end_frame}")
    adopt_arr[start_frame:end_frame, upper_left:upper_right] = T_arr[start_frame:end_frame, upper_left:upper_right]
    adopt_arr[start_frame:end_frame, lower_left:lower_right] = T_arr[start_frame:end_frame, lower_left:lower_right]


In [ ]:
# 確認
plt.imshow(adopt_arr, aspect='auto', cmap='jet')
plt.colorbar()
plt.show()
plt.close()

In [ ]:
# min, mean, maxを取り出す
df_T = pd.DataFrame(adopt_arr).T
df_T.replace([0], np.nan, inplace=True)

# 上流
df_T_up = df_T.iloc[0:256, :]
T_up = df_T_up.mean()
T_up_err = np.array([
    T_up - df_T_up.min(),
    df_T_up.max() - T_up,
])

# 下流
df_T_down = df_T.iloc[256:511, :]
T_down = df_T_down.mean()
T_down_err = np.array([
    T_down - df_T_down.min(),
    df_T_down.max() - T_down,
])

x = np.arange(frame_num)
plt.errorbar(x=x, y=T_up, yerr=T_up_err, c='blue', alpha=0.5)
plt.plot(df_T_up.min(), c='blue')
plt.plot(df_T_up.max(), c='blue')
plt.errorbar(x=x, y=T_down, yerr=T_down_err, c='green', alpha=0.5)
plt.plot(df_T_down.min(), c='green')
plt.plot(df_T_down.max(), c='green')
plt.show()
plt.close()

# XRDに合わせた配列を作る

In [ ]:
# nanを0に戻す
T_up = pd.DataFrame(T_up).replace(np.nan, 300).values
T_up_err = pd.DataFrame(T_up_err).replace(np.nan, 300).values
T_down = pd.DataFrame(T_down).replace(np.nan, 300).values
T_down_err = pd.DataFrame(T_down_err).replace(np.nan, 300).values

In [ ]:
# 補間するメソッド
from scipy.interpolate import interp1d

def interpolate_T_to_xrd(original_arr,
                         fps=fps, time_arr=time_arr, frame_num=frame_num,
                         xrd_fps=xrd_fps, xrd_frame_num=xrd_frame_num):
    T_fps = fps.copy()
    T_time_arr = time_arr.copy()
    
    new_frame_num = int(frame_num * (xrd_fps / T_fps))
    new_time_arr = np.arange(new_frame_num) / xrd_fps
    
    interpolator = interp1d(T_time_arr, original_arr, kind='linear', fill_value="extrapolate")
    new_arr = interpolator(new_time_arr)
    if new_frame_num < xrd_frame_num: # xrdのframeに合わせる
        delta = xrd_frame_num - new_frame_num
        new_arr = np.r_[new_arr, np.full(delta, 300)]
    return new_arr


In [ ]:
# 補間する
T_up_new = interpolate_T_to_xrd(T_up[:,0])
T_up_err_minus = interpolate_T_to_xrd(T_up_err[0])
T_up_err_plus = interpolate_T_to_xrd(T_up_err[1])

T_down_new = interpolate_T_to_xrd(T_down[:,0])
T_down_err_minus = interpolate_T_to_xrd(T_down_err[0])
T_down_err_plus = interpolate_T_to_xrd(T_down_err[1])

In [ ]:
# plotして確認
x = np.arange(xrd_frame_num)
plt.errorbar(x=x, y=T_up_new, yerr=[T_up_err_minus, T_up_err_plus], c='green', alpha=0.1)
plt.errorbar(x=x, y=T_down_new, yerr=[T_down_err_minus, T_down_err_plus], c='blue', alpha=0.1)
plt.ylim(1_000, 6_000) # 下は切ったほうがいい

In [ ]:
# hdfに書き込む
T_layer = 'entry/T/'

with h5py.File(hdf5_path, 'a') as f_append:
    f_append.create_dataset(name=T_layer + 'T_up', data=T_up_new)
    f_append.create_dataset(name=T_layer + 'T_down', data=T_down_new)
    
    f_append.create_dataset(name=T_layer + 'T_up_err_minus', data=T_up_err_minus)
    f_append.create_dataset(name=T_layer + 'T_up_err_plus', data=T_up_err_plus)
    f_append.create_dataset(name=T_layer + 'T_down_err_minus', data=T_down_err_minus)
    f_append.create_dataset(name=T_layer + 'T_down_err_plus', data=T_down_err_plus)

# 時間発展をplotする

In [ ]:
# saturation配列を作成する
saturation_boundaries = []
I_saturation = 65_000

if radiation.max() > I_saturation:
    adopt_flag = False
    for frame in range(frame_num):
        if adopt_flag is False: # 不採用 -> 採用
            if max_radi_frame_arr[frame] > I_saturation:
                print(frame, saturation_boundaries)
                saturation_boundaries.append(frame)
                adopt_flag = True
        else: # 採用 -> 不採用
            if max_radi_frame_arr[frame] < I_saturation:
                print(frame, saturation_boundaries)
                saturation_boundaries.append(frame)
                adopt_flag = False
    print(f"{saturation_boundaries}")
 
saturation_arr = np.array(saturation_boundaries)
saturation_time_arr = saturation_arr / fps
saturation_time_arr

In [ ]:
fig, ax = plt.subplots()

ax.set_title(f"T evolution of {run_name}")

x = np.arange(xrd_frame_num) / xrd_fps
ax.errorbar(x=x, y=T_up_new, yerr=[T_up_err_minus, T_up_err_plus],
            c='blue', marker='.',mfc='blue', capsize=3, elinewidth=1, alpha=0.3,
            label="Up")
ax.errorbar(x=x, y=T_down_new, yerr=[T_down_err_minus, T_down_err_plus],
            c='orange', marker='.',mfc='orange', capsize=3, elinewidth=1, alpha=0.3,
            label="down")

ax.set_ylim(1_000, 6_000)
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Temperature (K)')

In [ ]:
# satuあり、全体
fig, ax = plt.subplots()

ax.set_title(f"T evolution of {run_name}")

x = np.arange(xrd_frame_num) / xrd_fps
ax.errorbar(x=x, y=T_up_new, yerr=[T_up_err_minus, T_up_err_plus],
            c='blue', marker='.',mfc='white', capsize=3, elinewidth=1, alpha=0.5,
            label="Up")
ax.errorbar(x=x, y=T_down_new, yerr=[T_down_err_minus, T_down_err_plus],
            c='orange', marker='.',mfc='white', capsize=3, elinewidth=1, alpha=0.5,
            label="Down")

ax.set_ylim(1_000, 7_000)
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Temperature (K)')
ax.legend()

for i in range(region_num):
    idx = i * 2
    ax.axvspan(xmin=saturation_time_arr[idx], xmax=saturation_time_arr[idx+1],
               facecolor='red', alpha=0.3)


In [ ]:
# satuあり、一部
fig, ax = plt.subplots()

ax.set_title(f"T evolution of {run_name}")

x = np.arange(xrd_frame_num) / xrd_fps
ax.errorbar(x=x, y=T_up_new, yerr=[T_up_err_minus, T_up_err_plus],
            c='blue', marker='.',mfc='white', capsize=3, elinewidth=1, alpha=0.5,
            label="Up")
ax.errorbar(x=x, y=T_down_new, yerr=[T_down_err_minus, T_down_err_plus],
            c='orange', marker='.',mfc='white', capsize=3, elinewidth=1, alpha=0.5,
            label="Down")

ax.set_ylim(1_000, 7_000)
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Temperature (K)')
ax.legend()

for i in range(region_num):
    idx = i * 2
    ax.axvspan(xmin=saturation_time_arr[idx], xmax=saturation_time_arr[idx+1],
               facecolor='red', alpha=0.3)
ax.set_xlim(8.5, 9.5)

In [ ]:
# satuあり、全体、レーザーあり
fig, ax = plt.subplots()

ax.set_title(f"T evolution of {run_name}")

x = np.arange(xrd_frame_num) / xrd_fps
ax.errorbar(x=x, y=T_up_new, yerr=[T_up_err_minus, T_up_err_plus],
            c='blue', marker='.',mfc='white', capsize=3, elinewidth=1, alpha=0.5,
            label="Up")
ax.errorbar(x=x, y=T_down_new, yerr=[T_down_err_minus, T_down_err_plus],
            c='orange', marker='.',mfc='white', capsize=3, elinewidth=1, alpha=0.5,
            label="Down")

ax.set_ylim(1_000, 7_000)
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Temperature (K)')
ax.legend()

for i in range(region_num):
    idx = i * 2
    ax.axvspan(xmin=saturation_time_arr[idx], xmax=saturation_time_arr[idx+1],
               facecolor='red', alpha=0.3)

twin = ax.twinx()
twin.plot(laser_time_arr, laser_power_arr, color='black', alpha=0.3, label='Laser')
twin.legend(loc="upper left", bbox_to_anchor=(0, 0.8))

# 気になるところを見る

## 温度プロファイル

In [ ]:
# 気になるところの温度プロファイルをplotする
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

frames = range(360, 380, 1)
T_lim = [300, 6_000]

ax = axes[0]
for frame in frames:
    ax.plot(T_arr[frame, upper_left-10:upper_right+10], c='blue', alpha=0.5)
ax.set_ylim(T_lim[0], T_lim[1])

ax = axes[1]
for frame in frames:
    ax.plot(T_arr[frame, lower_left-10:lower_right+10], c='blue', alpha=0.5)
ax.set_ylim(T_lim[0], T_lim[1])


None

## スペクトルを見る

In [ ]:
# プランク関数の強度を返す関数
from scipy.constants import h, c, k

def planck_function(wavelengths, temperature):
    # 波長をメートルに変換 (nm -> m)
    wavelengths_m = np.array(wavelengths) * 1e-9

    # プランク関数の計算
    intensity = (2.0 * h * c**2) / (wavelengths_m**5) / (np.exp((h * c) / (wavelengths_m * k * temperature)) - 1.0)

    return intensity

In [ ]:
# unitの波長
wl_arr[300]

In [ ]:
# スペクトルと比較する
frames = range(360, 380, 1)
for frame in frames:
    unit = spectra[frame, lower_center, 300]
    if unit == 0:
        y = spectra[frame, lower_center, :]
    else:
        y = spectra[frame, lower_center, :] / unit
    plt.plot(wl_arr, y, c='blue', alpha=0.5)

In [ ]:
# 温度と合わせる
frames = range(360, 380, 1)
for frame in frames:
    # 実験
    unit = spectra[frame, lower_center, 300]
    if unit == 0:
        continue
    else:
        y = spectra[frame, lower_center, :] / unit
    plt.scatter(wl_arr, y, c='blue', alpha=0.5, s=1)
    # fitting
    T = T_arr[frame, lower_center]
    print(f"{frame = }, {round(T, 1) = } K")
    if T == 0:
        continue
    else:
        I = planck_function(wl_arr, T)
        y = I / I[300]
    plt.plot(wl_arr, y, c='red', alpha=0.5, linestyle='--')

In [ ]:
frames = range(360, 380, 1)
for frame in frames:
    plt.plot(wl_arr, spectra[frame, lower_center, :], c='blue', alpha=0.5)

## スペクトルをfittingする

In [ ]:
plt.plot(wl_arr)
plt.grid()

In [ ]:
from scipy.constants import h, c, k
from scipy.optimize import curve_fit

def planck_function(wavelengths, temperature, scale):
    # 波長をメートルに変換 (nm -> m)
    wavelengths_m = np.array(wavelengths) * 1e-9

    # プランク関数の計算
    intensity = (2.0 * h * c**2) / (wavelengths_m**5) / (np.exp((h * c) / (wavelengths_m * k * temperature)) - 1.0)

    return scale * intensity

def planck_fit(wavelengths, intensities):
    # フィッティング関数
    def fitting_func(wavelengths, temperature, scale):
        return planck_function(wavelengths, temperature, scale)
    
    initial_guess = [5000, 1e-9]  # 温度とスケールファクタの初期推定値
    
    # curve_fitでフィッティング
    popt, pcov = curve_fit(fitting_func, wavelengths, intensities, p0=initial_guess)
    
    # フィッティングされた温度とスケールファクタ、不確かさを返す
    T = popt[0]
    scale = popt[1]
    T_err = np.sqrt(pcov[0, 0])
    scale_err = np.sqrt(pcov[1, 1])
    
    # フィッティングされた温度を返す
    return [T, scale, T_err, scale_err]

### fitting場1

In [ ]:
# fitting

frame = 361

window = [110, 300] # 600-800

wls = wl_arr[window[0]:window[1]]
spectrum = spectra[frame, lower_center, window[0]:window[1]]

results_600_800 = planck_fit(wls, spectrum)
print(results_600_800)

# 違うとこ
window = [220, 300] # 700-800

wls = wl_arr[window[0]:window[1]]
spectrum = spectra[frame, lower_center, window[0]:window[1]]

results_700_800 = planck_fit(wls, spectrum)
print(results_700_800)

plt.plot(wls, spectrum)


In [ ]:
# 実験
unit = spectra[frame, lower_center, 300]
y = spectra[frame, lower_center, :] / unit
plt.scatter(wl_arr, y, c='blue', alpha=0.5, s=1)

# fitting
T = results_600_800[0]
scale = results_600_800[1]
print(f"{frame = }, {round(T, 1) = } K")
I = planck_function(wl_arr, T, scale)
y = I / I[300]
plt.plot(wl_arr, y, c='red', alpha=0.5, linestyle='-')

T = results_600_800[0] + results_600_800[2]
scale = results_600_800[1]
print(f"{frame = }, {round(T, 1) = } K")
I = planck_function(wl_arr, T, scale)
y = I / I[300]
plt.plot(wl_arr, y, c='red', alpha=0.5, linestyle='--')


T = results_700_800[0]
scale = results_700_800[1]
print(f"{frame = }, {round(T, 1) = } K")
I = planck_function(wl_arr, T, scale)
y = I / I[300]
plt.plot(wl_arr, y, c='orange', alpha=0.5, linestyle='--')


### fitting場1


In [ ]:
# fitting

frame = 367

window = [110, 300] # 600-800

wls = wl_arr[window[0]:window[1]]
spectrum = spectra[frame, lower_center, window[0]:window[1]]

results_600_800 = planck_fit(wls, spectrum)
print(results_600_800)

# 違うとこ
window = [220, 300] # 700-800

wls = wl_arr[window[0]:window[1]]
spectrum = spectra[frame, lower_center, window[0]:window[1]]

results_700_800 = planck_fit(wls, spectrum)
print(results_700_800)

plt.plot(wls, spectrum)


In [ ]:
# 実験
unit = spectra[frame, lower_center, 300]
y = spectra[frame, lower_center, :] / unit
plt.scatter(wl_arr, y, c='blue', alpha=0.5, s=1)

# fitting
T = results_600_800[0]
scale = results_600_800[1]
print(f"{frame = }, {round(T, 1) = } K")
I = planck_function(wl_arr, T, scale)
y = I / I[300]
plt.plot(wl_arr, y, c='red', alpha=0.5, linestyle='-')

T = results_600_800[0] + results_600_800[2]
scale = results_600_800[1]
print(f"{frame = }, {round(T, 1) = } K")
I = planck_function(wl_arr, T, scale)
y = I / I[300]
plt.plot(wl_arr, y, c='red', alpha=0.5, linestyle='--')


T = results_700_800[0]
scale = results_700_800[1]
print(f"{frame = }, {round(T, 1) = } K")
I = planck_function(wl_arr, T, scale)
y = I / I[300]
plt.plot(wl_arr, y, c='orange', alpha=0.5, linestyle='--')


In [ ]:
T = 2_000
scale = 1
print(f"{frame = }, {round(T, 1) = } K")
I = planck_function(wl_arr, T, scale)
y = I / I[300]
plt.plot(wl_arr, y, c='red', alpha=0.5, linestyle='--')
